In [1]:
#https://www.kaggle.com/code/taniaj/cryptocurrency-price-forecasting
#https://www.kaggle.com/code/shivamb/data-science-glossary-on-kaggle

In [2]:
import numpy as np 
import pandas as pd


eth = pd.read_csv('/kaggle/input/time-series-top-100-crypto-currency-dataset/ETH-USD.csv') # ETH coin data
bnb = pd.read_csv('/kaggle/input/time-series-top-100-crypto-currency-dataset/BNB-USD.csv') #binanace coin data
eth

,timestamp,adjclose,low,open,volume,close,high
0,2017-11-09 00:00:00,320.884003,307.056000,308.644989,893249984,320.884003,329.451996
1,2017-11-10 00:00:00,299.252991,294.541992,320.670990,885985984,299.252991,324.717987
2,2017-11-11 00:00:00,314.681000,298.191986,298.585999,842300992,314.681000,319.453003
3,2017-11-12 00:00:00,307.907990,298.513000,314.690002,1613479936,307.907990,319.153015
4,2017-11-13 00:00:00,316.716003,307.024994,307.024994,1041889984,316.716003,328.415009
...,...,...,...,...,...,...,...
1780,2022-09-24 00:00:00,1317.993286,1312.640259,1328.244629,12098209717,1317.993286,1346.255249
1781,2022-09-25 00:00:00,1294.216797,1275.627686,1317.938477,11802651633,1294.216797,1333.371338
1782,2022-09-26 00:00:00,1335.320190,1282.049316,1294.386108,16034549271,1335.320190,1335.526367
1783,2022-09-27 00:00:00,1330.127686,1308.991455,1335.337036,17870598937,1330.127686,1396.891479


# Data preporcessing


### Data cleaning

In [3]:
#missing values
eth.isnull().sum()

timestamp    0
adjclose     0
low          0
open         0
volume       0
close        0
high         0
dtype: int64

In [4]:
eth.describe()

,adjclose,low,open,volume,close,high
count,1785.000000,1785.000000,1785.000000,1.785000e+03,1785.000000,1785.000000
mean,1120.830330,1078.498985,1120.530123,1.302216e+10,1120.830330,1157.364326
std,1217.813261,1175.171508,1218.451695,1.092659e+10,1217.813261,1255.769033
min,84.308296,82.829887,84.279694,6.217330e+08,84.308296,85.342743
25%,207.082993,201.887238,207.100006,4.216140e+09,207.082993,211.778625
50%,462.436005,447.110992,462.437988,1.089479e+10,462.436005,473.558014
75%,1817.624146,1751.533325,1819.082031,1.833537e+10,1817.624146,1861.356934
max,4812.087402,4718.039062,4810.071289,8.448291e+10,4812.087402,4891.704590


In [5]:
t = pd.to_datetime(eth['timestamp'])

In [6]:
#Cecking the inconsistency in timestamp
# Calculate the difference between consecutive timestamps (in seconds)
time_deltas = t.diff().dt.total_seconds()
print(f"Time difference between timestamps (seconds):\n{time_deltas.describe()}")

Time difference between timestamps (seconds):
count      1784.000000
mean      86417.959641
std         758.568986
min       86400.000000
25%       86400.000000
50%       86400.000000
75%       86400.000000
max      118440.000000
Name: timestamp, dtype: float64


In [7]:
eth

,timestamp,adjclose,low,open,volume,close,high
0,2017-11-09 00:00:00,320.884003,307.056000,308.644989,893249984,320.884003,329.451996
1,2017-11-10 00:00:00,299.252991,294.541992,320.670990,885985984,299.252991,324.717987
2,2017-11-11 00:00:00,314.681000,298.191986,298.585999,842300992,314.681000,319.453003
3,2017-11-12 00:00:00,307.907990,298.513000,314.690002,1613479936,307.907990,319.153015
4,2017-11-13 00:00:00,316.716003,307.024994,307.024994,1041889984,316.716003,328.415009
...,...,...,...,...,...,...,...
1780,2022-09-24 00:00:00,1317.993286,1312.640259,1328.244629,12098209717,1317.993286,1346.255249
1781,2022-09-25 00:00:00,1294.216797,1275.627686,1317.938477,11802651633,1294.216797,1333.371338
1782,2022-09-26 00:00:00,1335.320190,1282.049316,1294.386108,16034549271,1335.320190,1335.526367
1783,2022-09-27 00:00:00,1330.127686,1308.991455,1335.337036,17870598937,1330.127686,1396.891479


In [8]:
correlation = eth.iloc[:,1:].corr(method='pearson')

In [9]:
correlation[['adjclose']].sort_values(['adjclose'], ascending=False)

,adjclose
adjclose,1.000000
close,1.000000
low,0.998935
high,0.998891
open,0.997809
volume,0.546383


### feature engineering

In [10]:
#feature selection
#Remove the close and as it is same as adjclose and we are considering adjclose also remove the 
#volume as it does not effect the the values of adjclose.
eth = eth.drop(['volume','close'],axis = 1)

In [11]:
eth

,timestamp,adjclose,low,open,high
0,2017-11-09 00:00:00,320.884003,307.056000,308.644989,329.451996
1,2017-11-10 00:00:00,299.252991,294.541992,320.670990,324.717987
2,2017-11-11 00:00:00,314.681000,298.191986,298.585999,319.453003
3,2017-11-12 00:00:00,307.907990,298.513000,314.690002,319.153015
4,2017-11-13 00:00:00,316.716003,307.024994,307.024994,328.415009
...,...,...,...,...,...
1780,2022-09-24 00:00:00,1317.993286,1312.640259,1328.244629,1346.255249
1781,2022-09-25 00:00:00,1294.216797,1275.627686,1317.938477,1333.371338
1782,2022-09-26 00:00:00,1335.320190,1282.049316,1294.386108,1335.526367
1783,2022-09-27 00:00:00,1330.127686,1308.991455,1335.337036,1396.891479


In [12]:
lag_val = 2
for col in eth.columns[1:]:
    for lag in range(1, lag_val+1):
        eth[f'{col}_lag{lag}'] = eth[col].shift(lag)
        

In [13]:
eth.head()

,timestamp,adjclose,low,open,high,adjclose_lag1,adjclose_lag2,low_lag1,low_lag2,open_lag1,open_lag2,high_lag1,high_lag2
0,2017-11-09 00:00:00,320.884003,307.056000,308.644989,329.451996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-11-10 00:00:00,299.252991,294.541992,320.670990,324.717987,320.884003,NaN,307.056000,NaN,308.644989,NaN,329.451996,NaN
2,2017-11-11 00:00:00,314.681000,298.191986,298.585999,319.453003,299.252991,320.884003,294.541992,307.056000,320.670990,308.644989,324.717987,329.451996
3,2017-11-12 00:00:00,307.907990,298.513000,314.690002,319.153015,314.681000,299.252991,298.191986,294.541992,298.585999,320.670990,319.453003,324.717987
4,2017-11-13 00:00:00,316.716003,307.024994,307.024994,328.415009,307.907990,314.681000,298.513000,298.191986,314.690002,298.585999,319.153015,319.453003


In [14]:
eth.isna().sum()

timestamp        0
adjclose         0
low              0
open             0
high             0
adjclose_lag1    1
adjclose_lag2    2
low_lag1         1
low_lag2         2
open_lag1        1
open_lag2        2
high_lag1        1
high_lag2        2
dtype: int64

In [15]:
eth = eth.dropna()

In [16]:
#this correlation is to see the correlation of lags witht eh adjclose value
correlation2 = eth.iloc[:,1:].corr(method='pearson')

In [17]:
correlation2[['adjclose']].sort_values(['adjclose'], ascending=False)

,adjclose
adjclose,1.000000
low,0.998934
high,0.998891
adjclose_lag1,0.997834
open,0.997808
high_lag1,0.996898
low_lag1,0.996677
adjclose_lag2,0.995952
open_lag1,0.995947
high_lag2,0.995009


### model building

In [18]:
eth.head()

,timestamp,adjclose,low,open,high,adjclose_lag1,adjclose_lag2,low_lag1,low_lag2,open_lag1,open_lag2,high_lag1,high_lag2
2,2017-11-11 00:00:00,314.681000,298.191986,298.585999,319.453003,299.252991,320.884003,294.541992,307.056000,320.670990,308.644989,324.717987,329.451996
3,2017-11-12 00:00:00,307.907990,298.513000,314.690002,319.153015,314.681000,299.252991,298.191986,294.541992,298.585999,320.670990,319.453003,324.717987
4,2017-11-13 00:00:00,316.716003,307.024994,307.024994,328.415009,307.907990,314.681000,298.513000,298.191986,314.690002,298.585999,319.153015,319.453003
5,2017-11-14 00:00:00,337.631012,316.763000,316.763000,340.177002,316.716003,307.907990,307.024994,298.513000,307.024994,314.690002,328.415009,319.153015
6,2017-11-15 00:00:00,333.356995,329.812988,337.963989,340.911987,337.631012,316.716003,316.763000,307.024994,316.763000,307.024994,340.177002,328.415009


In [19]:
eth = eth.drop('timestamp',axis = 1)

In [20]:
Y = eth['adjclose']
X = eth.drop('adjclose',axis = 1)

In [21]:
#scaling target feature
import sklearn
from sklearn.preprocessing import MinMaxScaler

def normalize_data(data):
    """
    Normalize the data using Min-Max scaling for each column.

    Parameters:
        data (DataFrame): Data to be normalized.

    Returns:
        normalized_data (DataFrame): Normalized data.
        scalers (dict): Dictionary containing scalers for each column.
    """
    scalers = {}
    normalized_data = data.copy()
    for column in data.columns:
        scaler = MinMaxScaler()
        normalized_column = scaler.fit_transform(data[[column]])
        normalized_data[column] = normalized_column
        scalers[column] = scaler
    return normalized_data, scalers


### split the data

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [23]:
# y_train = y_train.values
# y_test = y_test.values

In [24]:
#we have not used the the text data to avoid data leak
# X_train_normalized, X_scalers = normalize_data(X_train)

# y_scaler= MinMaxScaler()
# y_train_normalized = y_scaler.fit_transform(y_train.reshape(1, -1))

In [25]:
#now using the above scalers for training data and testing data lets normalize the test data
# for col in X_test.columns:
#     normalized = X_scalers[col].transform(X_test[[col]])
#     X_test[col] = normalized

# y_test_normalized = y_scaler.fit_transform(y_test.reshape(1, -1))

In [26]:
models_archetectures = [{'layers':(100,50,25),'activation':'relu'}, {'layers':(100,50,25),'activation':'tanh'}, {'layers':(100,50,25),'activation':'logistic'}]

In [27]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neural_network import MLPRegressor


for epoch in models_archetectures:
    mlp = MLPRegressor(hidden_layer_sizes=epoch['layers'], activation=epoch['activation'], random_state=42)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'mse = {mse}, mae = {mae}')
    break

mse = 1762.773171378, mae = 23.59082231137136


In [28]:
mlp

MLPRegressor(hidden_layer_sizes=(100, 50, 25), random_state=42)

In [29]:
y_pred

array([ 264.80036199, 4070.29809326,  461.65943477,  171.56601237,
        127.88815805,  360.28429951,  136.6646844 , 3700.57854325,
       3200.09104092,  848.00362706,  183.74340473,  376.77793439,
        748.34770037,  150.847275  , 1236.30653398, 1918.85511212,
        669.22059443, 3200.85421047, 1603.84741042, 1175.53843189,
        269.55116397,  307.05703718, 2582.35472821,  196.4049457 ,
        295.05815434,  200.85438702,  234.16983694, 2182.97367108,
        288.67673187,  142.24986578,  117.00365388, 1575.13327494,
       3258.92845479,  122.00220969,  584.9241466 , 3942.26398722,
       3450.60294417, 1322.28398619,  617.67990836,  116.2023839 ,
        183.54569043,  240.91566126,  459.43085617,  129.16895853,
        725.83628043, 3289.48733128,  140.87099481, 1822.548202  ,
       1978.38601233,  433.82740533,  491.32778949, 1034.07819946,
        616.54612509,  148.91990563, 3382.30648686,  365.52753817,
       1885.28082787, 1469.83126955,  604.33894407, 1986.65536

In [30]:
y_test

826      268.099243
1474    4088.457764
235      475.346985
800      175.365677
779      128.322708
           ...     
1520    3193.210449
1702    1222.506226
165      642.547974
671      178.725494
645      185.440079
Name: adjclose, Length: 446, dtype: float64

In [31]:
import joblib


In [32]:
filename = '/kaggle/working/my_mlp_model.pkl'
joblib.dump(mlp, filename)

['/kaggle/working/my_mlp_model.pkl']